# 10 SAXS Presets

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/08_workflow_system/10_saxs_presets.ipynb)

SAXS (Small-Angle X-ray Scattering) Domain Preset Example.

This example demonstrates how to create a custom preset for SAXS form factor
fitting using NLSQ's kwargs factory pattern.

SAXS analysis typically involves:
- Fitting form factor models (spheres, cylinders, core-shell, etc.)
- Parameters spanning many orders of magnitude (radius in nm, intensity in counts)
- High precision requirements for structural characterization
- Moderate dataset sizes (typically hundreds to thousands of q-points)

Run this example:
    python examples/scripts/08_workflow_system/10_saxs_presets.py

In [1]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

Not running in Colab - assuming NLSQ is already installed


In [2]:
import jax.numpy as jnp
import numpy as np

from nlsq import fit

In [3]:
def create_saxs_preset() -> dict:
    """Create a workflow configuration optimized for SAXS form factor fitting.

    SAXS-specific considerations:
    - Tolerances: 1e-9 for accurate structural parameters
    - Multi-start: Helpful for complex form factors with shape ambiguity
    - Normalization: Critical due to intensity/size parameter scale differences

    Returns
    -------
    dict
        kwargs for fit() optimized for SAXS analysis.

    Example
    -------
    >>> kwargs = create_saxs_preset()
    >>> kwargs['gtol']
    1e-09
    >>> kwargs['multistart']
    True
    """
    return {
        "workflow": "standard",
        "gtol": 1e-9,
        "ftol": 1e-9,
        "xtol": 1e-9,
        "multistart": True,
        "n_starts": 12,
        "sampler": "lhs",
    }


def sphere_form_factor(q, radius, scale, background):
    """Sphere form factor for SAXS fitting.

    The form factor amplitude for a uniform sphere:
        F(q) = 3 * [sin(qR) - qR*cos(qR)] / (qR)^3

    Intensity: I(q) = scale * |F(q)|^2 + background
    """
    qr = q * radius

    form_factor = jnp.where(
        jnp.abs(qr) < 1e-6,
        1.0 - qr**2 / 10.0,
        3.0 * (jnp.sin(qr) - qr * jnp.cos(qr)) / qr**3,
    )

    return scale * form_factor**2 + background


def core_shell_form_factor(q, r_core, r_shell, scale, background):
    """Core-shell sphere form factor for SAXS fitting."""

    def sphere_amplitude(q_val, r):
        qr = q_val * r
        return jnp.where(
            jnp.abs(qr) < 1e-6,
            1.0 - qr**2 / 10.0,
            3.0 * (jnp.sin(qr) - qr * jnp.cos(qr)) / qr**3,
        )

    v_core = (4.0 / 3.0) * jnp.pi * r_core**3
    v_shell = (4.0 / 3.0) * jnp.pi * r_shell**3

    f_core = v_core * sphere_amplitude(q, r_core)
    f_shell = v_shell * sphere_amplitude(q, r_shell)

    f_total = f_core + 0.5 * (f_shell - f_core)

    return scale * (f_total / v_shell) ** 2 + background

In [4]:
def main():
    print("=" * 70)
    print("SAXS Domain Preset Example")
    print("=" * 70)
    print()

    # Create the SAXS preset
    kwargs = create_saxs_preset()

    print("SAXS Preset Configuration:")
    print("-" * 40)
    for key, value in kwargs.items():
        print(f"  {key}: {value}")
    print()

    # Generate synthetic SAXS data for sphere form factor
    print("Generating synthetic SAXS data (spheres)...")
    np.random.seed(42)

    q_data = np.logspace(-2, 0, 200)

    true_params = {
        "radius": 10.0,
        "scale": 1e6,
        "background": 10.0,
    }

    y_true = sphere_form_factor(
        q_data,
        true_params["radius"],
        true_params["scale"],
        true_params["background"],
    )
    noise = 0.05 * np.sqrt(np.maximum(y_true, 1)) * np.random.randn(len(q_data))
    y_data = np.maximum(y_true + noise, 0.1)

    print(f"  Data points: {len(q_data)}")
    print(f"  True radius: {true_params['radius']:.2f} nm")
    print(f"  True scale:  {true_params['scale']:.2e}")
    print()

    p0 = [8.0, 5e5, 5.0]
    bounds = (
        [1.0, 1e3, 0.0],
        [50.0, 1e9, 100.0],
    )

    print("Fitting sphere form factor with SAXS preset...")
    popt, pcov = fit(
        sphere_form_factor,
        q_data,
        y_data,
        p0=p0,
        bounds=bounds,
        **kwargs,
    )

    print()
    print("Fit Results:")
    print("-" * 40)
    print(f"  radius:     {popt[0]:.4f} nm (true: {true_params['radius']:.4f})")
    print(f"  scale:      {popt[1]:.4e} (true: {true_params['scale']:.4e})")
    print(f"  background: {popt[2]:.4f} (true: {true_params['background']:.4f})")

    if pcov is not None:
        perr = np.sqrt(np.diag(pcov))
        print()
        print("Parameter Uncertainties (1-sigma):")
        print(f"  radius:     +/- {perr[0]:.4f} nm")
        print(f"  scale:      +/- {perr[1]:.4e}")
        print(f"  background: +/- {perr[2]:.4f}")

    rel_error = 100 * np.abs(popt[0] - true_params["radius"]) / true_params["radius"]
    print()
    print(f"  Radius relative error: {rel_error:.3f}%")

    print()
    print("=" * 70)
    print("Notes on SAXS Preset Customization")
    print("=" * 70)
    print()
    print("The SAXS preset uses the kwargs factory pattern with adjustments for:")
    print()
    print("1. Tighter tolerances (1e-9)")
    print("   - Form factor oscillations provide good gradient information")
    print("   - Accurate size determination requires higher precision")
    print("   - Still computationally efficient for typical SAXS datasets")
    print()
    print("2. Multi-start optimization (n_starts=12)")
    print("   - Helps with polydisperse systems")
    print("   - Useful for complex form factors (core-shell, ellipsoids)")
    print("   - Moderate number sufficient for well-conditioned problems")
    print()
    print("3. Parameter scaling considerations:")
    print("   - Intensity scales span many orders of magnitude")
    print("   - NLSQ's automatic normalization handles this well")
    print("   - Use bounds to constrain physically reasonable values")
    print()
    print("4. Common SAXS models:")
    print("   - Sphere, cylinder, ellipsoid form factors")
    print("   - Core-shell and multi-shell structures")
    print("   - Structure factors for concentrated systems")
    print()

In [5]:
if __name__ == "__main__":
    main()

SAXS Domain Preset Example

SAXS Preset Configuration:
----------------------------------------
  workflow: standard
  gtol: 1e-09
  ftol: 1e-09
  xtol: 1e-09
  multistart: True
  n_starts: 12
  sampler: lhs

Generating synthetic SAXS data (spheres)...


INFO:nlsq.multi_start:Generating 12 starting points using lhs n_starts=12 | sampler=lhs | center_on_p0=True


  Data points: 200
  True radius: 10.00 nm
  True scale:  1.00e+06

Fitting sphere form factor with SAXS preset...


INFO:nlsq.multi_start:Evaluating 12 starting points


INFO:nlsq.curve_fit:Starting curve fit n_params=3 | n_data_points=200 | method=trf | solver=auto | batch_size=None | has_bounds=True | dynamic_sizing=False


...

INFO:nlsq.least_squares:Convergence reason=`xtol` termination condition is satisfied. | iterations=23 | final_cost=1.3049e+05 | elapsed=0.211s | final_gradient_norm=239.6761


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit elapsed=0.324374s


INFO:nlsq.curve_fit:Curve fit completed total_time=0.3244 | final_cost=2.6098e+05 | covariance_warning=False


INFO:nlsq.multi_start:Best starting point: loss=260982.042384 best_loss=2.6098e+05 | best_params=[9.999965073604928, 999994.8064380719, 9.73953698326803]



Fit Results:
----------------------------------------
  radius:     10.0000 nm (true: 10.0000)
  scale:      9.9999e+05 (true: 1.0000e+06)
  background: 9.7395 (true: 10.0000)

Parameter Uncertainties (1-sigma):
  radius:     +/- 0.0001 nm
  scale:      +/- 6.4071e+00
  background: +/- 5.4235

  Radius relative error: 0.000%

Notes on SAXS Preset Customization

The SAXS preset uses the kwargs factory pattern with adjustments for:

1. Tighter tolerances (1e-9)
   - Form factor oscillations provide good gradient information
   - Accurate size determination requires higher precision
   - Still computationally efficient for typical SAXS datasets

2. Multi-start optimization (n_starts=12)
   - Helps with polydisperse systems
   - Useful for complex form factors (core-shell, ellipsoids)
   - Moderate number sufficient for well-conditioned problems

3. Parameter scaling considerations:
   - Intensity scales span many orders of magnitude
   - NLSQ's automatic normalization handles this well
 